In [1]:
import scipy.stats as stats
import subprocess
import scipy as sp
import rasterio
import numpy as np
import meshio
import pandas as pd
from matplotlib import pyplot as plt
import pyvista
meshC =  pyvista.read(r"C:\Users\Andrea\Documents\Flow\OpenGeoSys\ogs6hetPermAquiferLim13-19\gridTetraCoarse.vtu")

In [2]:

df = pd.read_csv('C:\\Users\\Andrea\\Documents\\Flow\\Optimization\\headData\\averageInTime\\wellHeadPo2013_2020.csv')
wellLoc = df
wellLoc['Bottom'] = wellLoc['quota[m]']-wellLoc['depth']

weights = np.loadtxt('weights.txt')

nRecostruction = sum(weights[:,1])


def flowSimulation(P):
    #model parameters
    
    myfile = open(r"C:\Users\Andrea\Documents\Flow\OpenGeoSys\ogs6hetPermAquiferLim13-19\Po1HP.prj", "rt")
    contents = myfile.readlines()
    myfile.close()
    
    wordX = 'permeability'
    nameOfRivers = ['Adige','Secchia','DoraBaltea','Adda','Ticino','Chiese','Oglio','Tanaro','Po1','Po2','Po3','Po4','Po5','Po6','Reno','Lamone','Savio','none']
    al = 0
    worda = 'alpha' + nameOfRivers[al]
    wordqm = 'qms'
    wordh0 = 'h0'
    j=0
    qms = 0
    h0 = 0
    P0 = h0*9810
    
    # set the K parameters 
    Kappa1 = (10**P[0]*weights[0,:] + 10**P[1]*weights[1,:] + 10**P[2]*weights[2,:] + 10**P[3]*weights[3,:] + 10**P[4]*weights[4,:] + 10**P[5]*weights[5,:])/nRecostruction
    meshC['Kappa1'] = Kappa1
    meshC.save(r"C:\Users\Andrea\Documents\Flow\OpenGeoSys\ogs6hetPermAquiferLim13-19\gridTetraCoarse.vtu")
    for i in range(len(contents)):
#         if wordX in contents[i]:
#             value = contents[i+2].split('<value>').pop().split('</value>')[0]
#             if j>0:
#                 K = 10**P[j-1]
#                 value2= str(K)
#                 contents[i+2] = contents[i+2].replace(value, value2)
#                 permName = 'k%1d' % j 
#                 print(permName, value2)
#             j += 1

    
    
    
        #set the alpha values
        if worda in contents[i]:
            if '<value>' in contents[i+2]:
                value = contents[i+2].split('<value>').pop().split('</value>')[0]
                a = P[al+6]
                value2= str(a)
                contents[i+2] = contents[i+2].replace(value, value2)
                print(worda, value2)
                al += 1
                worda = 'alpha' + nameOfRivers[al] 
                
                
                
               
        #set the qm values          
        if wordqm in contents[i]:
              if '<value>' in contents[i+2]:
                    value = contents[i+2].split('<value>').pop().split('</value>')[0]
                    qmsValue2 = P[-1]*float(value)
                    value2= str(qmsValue2)
                    contents[i+2] = contents[i+2].replace(value, value2)                    
                    print('C_qms', qms+1 ,':', P[-1],';', 'qms', qms+1 , ':' , value2)
                    qms += 1
        if wordh0 in contents[i]:
              if '<value>' in contents[i+2]:
                    value = contents[i+2].split('<value>').pop().split('</value>')[0]
                    value2= str(P0)
                    contents[i+2] = contents[i+2].replace(value, value2)                    
    
    
            
    with open(r"C:\Users\Andrea\Documents\Flow\OpenGeoSys\ogs6hetPermAquiferLim13-19\Po2.prj", 'w') as file:
        file.writelines( contents )
    
    
    subprocess.check_call([r"C:\Users\Andrea\Documents\Flow\OpenGeoSys\ogs-6.4.3-16-g9a7564950-Windows-10.0.22000-python-3.10.7-utils\bin\ogs.exe", r"C:\Users\Andrea\Documents\Flow\OpenGeoSys\ogs6hetPermAquiferLim13-19\Po2.prj"])

    mesh = meshio.read(r"C:\Users\Andrea\Documents\Flow\OpenGeoSys\ogs6hetPermAquiferLim13-19\Po_ts_1_t_1.000000.vtu")
    data = mesh.points
    sample = np.array([wellLoc.longitude,wellLoc.latitude,wellLoc.Bottom]).T
    kdtree=sp.spatial.KDTree(data)
    dist,points=kdtree.query(sample,1)
    simulatedH = (mesh.point_data['pressure'][points].T)/9810+sample[:,2]
    fobj = (sum(((simulatedH-np.array(wellLoc['head[m]']))**2)**(1/2)))/len(wellLoc['head[m]'])
    print('RMSE: ',fobj)
    return fobj

In [3]:
print(meshC)

UnstructuredGrid (0x1d46c22bca0)
  N Cells:	67959
  N Points:	36036
  X Bounds:	6.022e+05, 1.050e+06
  Y Bounds:	5.449e+06, 5.631e+06
  Z Bounds:	-3.500e+02, 6.044e+02
  N Arrays:	3



In [4]:
P = [ -8.72790795,    #log_10(k1)
     -8.59778023,     #log_10(k2)
     -8.26612873,   #log_10(k3)
     -9.76895812,    #log_10(k4)
     -12.52132462,    #log_10(k5)
     -15.84890911,     #log_10(k6)
     7.094148178760728e-06,  #alpha
     1.86339251,     #C_qms1-29
    ]

 x1: array([ -8.72790795,  -8.59778023,  -8.26612873,  -9.76895812,
       -12.52132462, -15.84890911,   1.86339251])
      x2: array([1.49797686e-05, 8.10629932e-05, 8.04589383e-06, 3.17875276e-08,
       4.48833080e-05, 7.55152160e-05, 6.77841116e-05, 4.46926834e-05,
       4.91272024e-05, 8.46664613e-05, 3.50995019e-05, 4.80618150e-05,
       6.01917886e-05, 5.57197705e-05, 5.62985577e-05, 4.68726223e-05,
       8.10237680e-05])


IndentationError: unexpected indent (238016746.py, line 11)

In [ ]:
#2run the model with the parameters set in P
flowSimulation(P)

In [ ]:
#plot the error  in a map
mesh = meshio.read(r"C:\Users\Andrea\Documents\Flow\OpenGeoSys\ogs6hetPermAquiferLim13-19\Po_ts_1_t_1.000000.vtu")
data = mesh.points
sample = np.array([wellLoc.longitude,wellLoc.latitude,wellLoc.Bottom]).T
kdtree=sp.spatial.KDTree(data)
dist,points=kdtree.query(sample,1)
simulatedH = (mesh.point_data['pressure'][points].T)/9810+sample[:,2]

fobj = (sum(((simulatedH-np.array(wellLoc['head[m]']))**2)**(1/2)))/len(wellLoc['head[m]'])
print('fobj:',fobj)
err = simulatedH-np.array(wellLoc['head[m]'])
img = plt.imread(r"C:\Users\Andrea\Documents\Flow\Optimization\background.tif")
img = np.array(img)
img[:,:,3] = img[:,:,3]*0.32
ext = [540000.0000, 1100000.0000,5400000.0000,5740000.0000]
plt.imshow(img, zorder=0, extent=ext)
plt.scatter(sample[:,0],sample[:,1],zorder=1, s=10, c=err, cmap='gist_rainbow', vmin=min(err), vmax=max(err))#vmin=min(err), vmax=max(err))
plt.colorbar()
plt.title('simulated-measure')
plt.show()

In [ ]:
counts, bins = np.histogram(err,bins=100, density=True)
plt.stairs(counts, bins,fill=True)
plt.xlabel('err = f_simulated-f_measured')
plt.axvline(x = np.average(err), color = 'g', label = 'Average error')
plt.axvline(x = np.average(err) + np.std(err), color = 'r', label = 'Avg + std')
plt.axvline(x = np.average(err) - np.std(err), color = 'r', label = 'Avg - std')
mu = np.average(err)
sigma =  np.std(err)
x = np.linspace(mu - 5*sigma, mu + 5*sigma, 100)
plt.plot(x, stats.norm.pdf(x, mu, sigma),label = 'Normal Distribution')
plt.legend()
print('Average error:', mu)
print('error standard deviation:', sigma)
